In [1]:
try:
    # On Google Colab
    from google.colab import drive

    drive.mount("/content/drive")

    BASEDIR = (
        "drive/MyDrive/DataAnalysis/相模湖・津久井湖土地・水質/yamanashi-water/model2024/"
    )
    ! pip install japanize_matplotlib
except:
    BASEDIR = "./"


In [22]:
import pandas as pd
import numpy as np


# read_csvが読み損じるので、自分で読む。
def read_csv(filename, sep=",", encoding="utf8"):
    rows = []
    with open(filename, encoding=encoding) as f:
        for line in f.readlines():
            cols = line.split(sep)[:143]
            rows.append(cols)
    df = pd.DataFrame(rows[1:])
    df.columns = rows[0][:143]
    print(rows[0][:143])
    return df


cols = """月平均値(ppm)_４月
同左_５月.2
同左_６月.2
同左_７月.2
同左_８月.2
同左_９月.2
同左_10月.2
同左_11月.2
同左_12月.2
同左_１月.2
同左_２月.2
同左_３月.2
""".splitlines()

# 2008年のデータはコラム名をつけまちがっている!!!信じられん。

stations = [19202030, 19204010, 19206010]

nox = []
for year in range(2000, 2021):
    if year >= 2003:
        cols = "月平均値(ppm)_４月,月平均値(ppm)_５月,月平均値(ppm)_６月,月平均値(ppm)_７月,月平均値(ppm)_８月,月平均値(ppm)_９月,月平均値(ppm)_10月,月平均値(ppm)_11月,月平均値(ppm)_12月,月平均値(ppm)_１月,月平均値(ppm)_２月,月平均値(ppm)_３月".split(
            ","
        )
    filename = f"{BASEDIR}../TD20231218_155555_51/TD{year}0419.txt"
    cols_in_file = list(
        pd.read_csv(
            filename,
            sep=",",
            nrows=1,
            encoding="cp932",
        )
    )
    print(len(cols_in_file))
    df = pd.read_csv(
        filename,
        encoding="cp932",
        sep=",",
        usecols=cols_in_file,
    )
    df = df.loc[:, ~df.columns.duplicated()]
    print(year, df)
    for i, col in enumerate(cols):
        month = i + 4
        y = year
        if month > 12:
            y += 1
            month -= 12
        row = []
        for station in stations:
            rows = df[df["測定局コード"] == station]
            if len(rows) > 0:
                row.append(rows.iloc[0][col])
            else:
                row.append(np.NaN)
        print(row)
        nox.append([float(x) for x in row])

nox = pd.DataFrame(nox, columns=stations)

nox

175
2000     測定年度  項目種類コード  項目コード_数字 項目コード_英数字  測定方法コード  都道府県コード 都道府県名        同左_ローマ字  \
0   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
1   2000        1         4       NOX        2       19   山梨県  Yamanashi-keｎ   
2   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
3   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
4   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
5   2000        1         4       NOX        2       19   山梨県  Yamanashi-keｎ   
6   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
7   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
8   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
9   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
10  2000        1         4       NOX        2       19   山梨県  Yamanashi-keｎ   

    市区町村コード  市区町村名  ... Unname

,19202030,19204010,19206010
0,0.011,0.024,0.030
1,0.009,0.027,0.024
2,0.011,0.029,0.027
3,0.010,0.023,0.023
4,0.011,0.023,0.022
...,...,...,...
247,0.007,0.009,0.023
248,0.009,0.011,NaN
249,0.010,0.013,0.028
250,0.007,0.010,0.020


月付けを追加する。

In [25]:
nox["datetime"] = pd.date_range(
    start="2000/04/01", freq="MS", periods=12 * 21
).tz_localize("Asia/Tokyo")
nox

,19202030,19204010,19206010,datetime
0,0.011,0.024,0.030,2000-04-01 00:00:00+09:00
1,0.009,0.027,0.024,2000-05-01 00:00:00+09:00
2,0.011,0.029,0.027,2000-06-01 00:00:00+09:00
3,0.010,0.023,0.023,2000-07-01 00:00:00+09:00
4,0.011,0.023,0.022,2000-08-01 00:00:00+09:00
...,...,...,...,...
247,0.007,0.009,0.023,2020-11-01 00:00:00+09:00
248,0.009,0.011,NaN,2020-12-01 00:00:00+09:00
249,0.010,0.013,0.028,2021-01-01 00:00:00+09:00
250,0.007,0.010,0.020,2021-02-01 00:00:00+09:00


新しい日時を1時間ごとに作る。

In [27]:
dates = pd.date_range("2000/4/1", "2021/3/31", freq="H").tz_localize("Asia/Tokyo")
ts = pd.DataFrame()
ts["datetime"] = dates
ts  # dates

/var/folders/g2/t_qwkbf48xj4860059bm17mh0000gs/T/ipykernel_65122/3915294546.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range("2000/4/1", "2021/3/31", freq='H').tz_localize("Asia/Tokyo")


,datetime
0,2000-04-01 00:00:00+09:00
1,2000-04-01 01:00:00+09:00
2,2000-04-01 02:00:00+09:00
3,2000-04-01 03:00:00+09:00
4,2000-04-01 04:00:00+09:00
...,...
184052,2021-03-30 20:00:00+09:00
184053,2021-03-30 21:00:00+09:00
184054,2021-03-30 22:00:00+09:00
184055,2021-03-30 23:00:00+09:00


In [28]:
pd.merge(ts, nox, on="datetime", how="outer")

,datetime,19202030,19204010,19206010
0,2000-04-01 00:00:00+09:00,0.011,0.024,0.03
1,2000-04-01 01:00:00+09:00,NaN,NaN,NaN
2,2000-04-01 02:00:00+09:00,NaN,NaN,NaN
3,2000-04-01 03:00:00+09:00,NaN,NaN,NaN
4,2000-04-01 04:00:00+09:00,NaN,NaN,NaN
...,...,...,...,...
184052,2021-03-30 20:00:00+09:00,NaN,NaN,NaN
184053,2021-03-30 21:00:00+09:00,NaN,NaN,NaN
184054,2021-03-30 22:00:00+09:00,NaN,NaN,NaN
184055,2021-03-30 23:00:00+09:00,NaN,NaN,NaN


In [30]:
nox = pd.merge(ts, nox, on="datetime", how="outer").ffill()
nox

,datetime,19202030,19204010,19206010
0,2000-04-01 00:00:00+09:00,0.011,0.024,0.030
1,2000-04-01 01:00:00+09:00,0.011,0.024,0.030
2,2000-04-01 02:00:00+09:00,0.011,0.024,0.030
3,2000-04-01 03:00:00+09:00,0.011,0.024,0.030
4,2000-04-01 04:00:00+09:00,0.011,0.024,0.030
...,...,...,...,...
184052,2021-03-30 20:00:00+09:00,0.006,0.007,0.015
184053,2021-03-30 21:00:00+09:00,0.006,0.007,0.015
184054,2021-03-30 22:00:00+09:00,0.006,0.007,0.015
184055,2021-03-30 23:00:00+09:00,0.006,0.007,0.015


In [1]:
from TD20231218_155555_51.reader import read

read(".")

175
2000     測定年度  項目種類コード  項目コード_数字 項目コード_英数字  測定方法コード  都道府県コード 都道府県名        同左_ローマ字  \
0   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
1   2000        1         4       NOX        2       19   山梨県  Yamanashi-keｎ   
2   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
3   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
4   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
5   2000        1         4       NOX        2       19   山梨県  Yamanashi-keｎ   
6   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
7   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
8   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
9   2000        1         4       NOX        1       19   山梨県  Yamanashi-keｎ   
10  2000        1         4       NOX        2       19   山梨県  Yamanashi-keｎ   

    市区町村コード  市区町村名  ... Unname

/Users/matto/My Drive/DataAnalysis/相模湖・津久井湖土地・水質/yamanashi-water/TD20231218_155555_51/read.py:86: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range("2000/4/1", "2021/3/31", freq="H").tz_localize("Asia/Tokyo")


,datetime,19202030,19204010,19206010
0,2000-04-01 00:00:00+09:00,0.011,0.024,0.030
1,2000-04-01 01:00:00+09:00,0.011,0.024,0.030
2,2000-04-01 02:00:00+09:00,0.011,0.024,0.030
3,2000-04-01 03:00:00+09:00,0.011,0.024,0.030
4,2000-04-01 04:00:00+09:00,0.011,0.024,0.030
...,...,...,...,...
184052,2021-03-30 20:00:00+09:00,0.006,0.007,0.015
184053,2021-03-30 21:00:00+09:00,0.006,0.007,0.015
184054,2021-03-30 22:00:00+09:00,0.006,0.007,0.015
184055,2021-03-30 23:00:00+09:00,0.006,0.007,0.015
